In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
# import torch.distributions as dist
import os
import shutil
import argparse
from tqdm import tqdm
import time
from collections import defaultdict
import pandas as pd
from utils.checkpoints import CheckpointIO
from utils.io import export_pointcloud
from utils.visualize import visualize_data
from utils.mesh import VoxelGrid
import yaml
from data import dataset
from models import OccupancyNetwork
from mesh_generate import generate


In [3]:
path="generation.yaml"
with open(path, 'r') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

out_dir = cfg['training']['out_dir']
generation_dir = os.path.join(out_dir, cfg['generation']['generation_dir'])
out_time_file = os.path.join(generation_dir, 'time_generation_full.pkl')
out_time_file_class = os.path.join(generation_dir, 'time_generation.pkl')

batch_size = cfg['generation']['batch_size']
input_type = cfg['data']['input_type']
vis_n_outputs = cfg['generation']['vis_n_outputs']
if vis_n_outputs is None:
    vis_n_outputs = -1

In [4]:
print(device)

cuda


In [5]:
# Release all unoccupied cached memory
import gc
torch.cuda.empty_cache()
gc.collect()
torch.backends.cuda.max_split_size_mb = 1024*4 

In [6]:
# Dataset
dataset = dataset.get_dataset('test', cfg, return_idx=True)

# Model
model = OccupancyNetwork.OccupancyNetwork({
    'img_size' : 224,
    'batch_size': 128,
    'input_image_channels': 3,
    'n_patches': 28,
    'n_blocks': 8,
    'hidden_d': 64,
    'n_heads': 2,
    'out_d': 256,
    'mlp_ratio': 4
}, device=device)

#
# x=cfg['test']['model_file']
# print(os.path.exists(x))
checkpoint_io = CheckpointIO(out_dir, model=model)
checkpoint_io.load("out/onet/model.pt")

# Generator
generator = generate.Generator3D(
        model,
        device=device,
        threshold=cfg['test']['threshold'],
        resolution0=cfg['generation']['resolution_0'],
        upsampling_steps=cfg['generation']['upsampling_steps'],
        #upsampling_steps=0,
        sample=cfg['generation']['use_sampling'],
        refinement_step=cfg['generation']['refinement_step'],
        simplify_nfaces=cfg['generation']['simplify_nfaces'],
        preprocessor=None
    )

# Determine what to generate
generate_mesh = cfg['generation']['generate_mesh']
generate_pointcloud = cfg['generation']['generate_pointcloud']
print(generate_pointcloud)

if generate_mesh and not hasattr(generator, 'generate_mesh'):
    generate_mesh = False
    print('Warning: generator does not support mesh generation.')

if generate_pointcloud and not hasattr(generator, 'generate_pointcloud'):
    generate_pointcloud = False
    print('Warning: generator does not support pointcloud generation.')



out/onet/model.pt
=> Loading checkpoint from local file...
True


In [7]:
print(dataset)

In [8]:
# Loader
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, num_workers=0, shuffle=False)

# Statistics
time_dicts = []

# Generate
model.eval()

# Count how many models already created
model_counter = defaultdict(int)

In [9]:
torch.cuda.empty_cache()


In [10]:
for it, data in enumerate(tqdm(test_loader)):
    # Output folders
    mesh_dir = os.path.join(generation_dir, 'meshes')
    pointcloud_dir = os.path.join(generation_dir, 'pointcloud')
    in_dir = os.path.join(generation_dir, 'input')
    generation_vis_dir = os.path.join(generation_dir, 'vis', )

    # Get index etc.
    idx = data['idx'].item()

    try:
        model_dict = dataset.get_model_dict(idx)
    except AttributeError:
        model_dict = {'model': str(idx), 'category': 'n/a'}
    
    modelname = model_dict['model']
    category_id = model_dict.get('category', 'n/a')

    try:
        category_name = dataset.metadata[category_id].get('name', 'n/a')
    except AttributeError:
        category_name = 'n/a'

    if category_id != 'n/a':
        mesh_dir = os.path.join(mesh_dir, str(category_id))
        pointcloud_dir = os.path.join(pointcloud_dir, str(category_id))
        in_dir = os.path.join(in_dir, str(category_id))

        folder_name = str(category_id)
        if category_name != 'n/a':
            folder_name = str(folder_name) + '_' + category_name.split(',')[0]

        generation_vis_dir = os.path.join(generation_vis_dir, folder_name)

    # Create directories if necessary
    if vis_n_outputs >= 0 and not os.path.exists(generation_vis_dir):
        os.makedirs(generation_vis_dir)

    if generate_mesh and not os.path.exists(mesh_dir):
        os.makedirs(mesh_dir)

    if generate_pointcloud and not os.path.exists(pointcloud_dir):
        os.makedirs(pointcloud_dir)

    if not os.path.exists(in_dir):
        os.makedirs(in_dir)
    
    # Timing dict
    time_dict = {
        'idx': idx,
        'class id': category_id,
        'class name': category_name,
        'modelname': modelname,
    }
    time_dicts.append(time_dict)

    # Generate outputs
    out_file_dict = {}

    # Also copy ground truth
    if cfg['generation']['copy_groundtruth']:
        modelpath = os.path.join(
            dataset.dataset_folder, category_id, modelname, 
            cfg['data']['watertight_file'])
        out_file_dict['gt'] = modelpath

    if generate_mesh:
        t0 = time.time()
        out = generator.generate_mesh(data)
        time_dict['mesh'] = time.time() - t0

        # Get statistics
        try:
            mesh, stats_dict = out
        except TypeError:
            mesh, stats_dict = out, {}
        time_dict.update(stats_dict)

        # Write output
        mesh_out_file = os.path.join(mesh_dir, '%s.off' % modelname)
        mesh.export(mesh_out_file)
        out_file_dict['mesh'] = mesh_out_file

    if generate_pointcloud:
        t0 = time.time()
        pointcloud = generator.generate_pointcloud(data)
        time_dict['pcl'] = time.time() - t0
        pointcloud_out_file = os.path.join(
            pointcloud_dir, '%s.ply' % modelname)
        export_pointcloud(pointcloud, pointcloud_out_file)
        out_file_dict['pointcloud'] = pointcloud_out_file

    if cfg['generation']['copy_input']:
        # Save inputs
        if input_type == 'img':
            inputs_path = os.path.join(in_dir, '%s.jpg' % modelname)
            inputs = data['inputs'].squeeze(0).cpu()
            visualize_data(inputs, 'img', inputs_path)
            out_file_dict['in'] = inputs_path
        elif input_type == 'voxels':
            inputs_path = os.path.join(in_dir, '%s.off' % modelname)
            inputs = data['inputs'].squeeze(0).cpu()
            voxel_mesh = VoxelGrid(inputs).to_mesh()
            voxel_mesh.export(inputs_path)
            out_file_dict['in'] = inputs_path
        elif input_type == 'pointcloud':
            inputs_path = os.path.join(in_dir, '%s.ply' % modelname)
            inputs = data['inputs'].squeeze(0).cpu().numpy()
            export_pointcloud(inputs, inputs_path, False)
            out_file_dict['in'] = inputs_path

    # Copy to visualization directory for first vis_n_output samples
    c_it = model_counter[category_id]
    if c_it < vis_n_outputs:
        # Save output files
        img_name = '%02d.off' % c_it
        for k, filepath in out_file_dict.items():
            ext = os.path.splitext(filepath)[1]
            out_file = os.path.join(generation_vis_dir, '%02d_%s%s'
                                    % (c_it, k, ext))
            shutil.copyfile(filepath, out_file)

    model_counter[category_id] += 1




100%|█████████████████████████████████████████████████████████████████████████████████| 8751/8751 [1:49:02<00:00,  1.34it/s]


In [11]:
# Create pandas dataframe and save
time_df = pd.DataFrame(time_dicts)
time_df.set_index(['idx'], inplace=True)
time_df.to_pickle(out_time_file)
print(time_df)


      class id                class name  \
idx                                        
0     02691156  airplane,aeroplane,plane   
1     02691156  airplane,aeroplane,plane   
2     02691156  airplane,aeroplane,plane   
3     02691156  airplane,aeroplane,plane   
4     02691156  airplane,aeroplane,plane   
...        ...                       ...   
8746  03001627                     chair   
8747  03001627                     chair   
8748  03001627                     chair   
8749  03001627                     chair   
8750  03001627                     chair   

                                  modelname      mesh  time (encode inputs)  \
idx                                                                           
0          d18592d9615b01bbbc0909d98a1ff2b4  3.130261              1.611435   
1          d18f2aeae4146464bd46d022fd7d80aa  0.293366              0.030574   
2          d199612c22fe9313f4fb6842b3610149  0.332376              0.020477   
3          d1a887a47991d1b3bc090

In [12]:

# Create pickle files  with main statistics
time_df_class = time_df.drop('modelname', axis=1).groupby(by=['class name']).mean()

# Print the time_df_class dataframe
time_df_class.to_pickle(out_time_file_class)

# Print results
time_df_class.loc['mean'] = time_df_class.mean()
print('Timings [s]:')
print(time_df_class)

Timings [s]:
                                                   class id      mesh  \
class name                                                              
airplane,aeroplane,plane                                inf  0.312784   
bench                                                   inf  0.435946   
cabinet                                                 inf  0.754491   
car,auto,automobile,machine,motorcar                    inf  0.401815   
chair                                                   inf  0.710786   
display,video display                                   inf  0.727742   
lamp                                                    inf  0.413744   
loudspeaker,speaker,speaker unit,loudspeaker sy...      inf  1.103695   
rifle                                                   inf  0.271617   
sofa,couch,lounge                                       inf  0.616307   
table                                                   inf  0.736627   
telephone,phone,telephone set         